In [2]:
import pandas as pd
from pandas import HDFStore
import numpy as np
import matplotlib.pyplot as plt
import math
import time
from sklearn import tree
import seaborn as sns
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing

%matplotlib inline

D:\Anaconda\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
y2015 = pd.read_csv('F:/Thinkful/Files/Loan Data 2015.csv')

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

id
421097
term
2
int_rate
110
grade
7
sub_grade
35
emp_title
120812
emp_length
11
home_ownership
4
verification_status
3
issue_d
12
loan_status
7
pymnt_plan
1
url
421095
desc
34
purpose
14
title
27
zip_code
914
addr_state
49
earliest_cr_line
668
revol_util
1211
initial_list_status
2
last_pymnt_d
25
next_pymnt_d
4
last_credit_pull_d
26
application_type
2
verification_status_joint
3


In [5]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

In [6]:
y2015.tail()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
421092,36271333.0,38982739.0,13000.0,13000.0,13000.0,60 months,15.99,316.07,D,5 years,...,0.0,3.0,100.0,50.0,1.0,0.0,51239.0,34178.0,10600.0,33239.0
421093,36490806.0,39222577.0,12000.0,12000.0,12000.0,60 months,19.99,317.86,E,1 year,...,1.0,2.0,95.0,66.7,0.0,0.0,96919.0,58418.0,9700.0,69919.0
421094,36271262.0,38982659.0,20000.0,20000.0,20000.0,36 months,11.99,664.20,B,10+ years,...,0.0,1.0,100.0,50.0,0.0,1.0,43740.0,33307.0,41700.0,0.0
421095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

In [8]:
pd.get_dummies(y2015)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,last_credit_pull_d_Nov-2016,last_credit_pull_d_Oct-2015,last_credit_pull_d_Oct-2016,last_credit_pull_d_Sep-2015,last_credit_pull_d_Sep-2016,application_type_INDIVIDUAL,application_type_JOINT,verification_status_joint_Not Verified,verification_status_joint_Source Verified,verification_status_joint_Verified
0,68009401.0,72868139.0,16000.0,16000.0,16000.0,14.85,379.39,48000.0,33.18,0.0,...,0,0,0,0,0,1,0,0,0,0
1,68354783.0,73244544.0,9600.0,9600.0,9600.0,7.49,298.58,60000.0,22.44,0.0,...,0,0,0,0,0,1,0,0,0,0
2,68466916.0,73356753.0,25000.0,25000.0,25000.0,7.49,777.55,109000.0,26.02,0.0,...,0,0,0,0,0,1,0,0,0,0
3,68466961.0,73356799.0,28000.0,28000.0,28000.0,6.49,858.05,92000.0,21.60,0.0,...,0,0,0,0,0,1,0,0,0,0
4,68495092.0,73384866.0,8650.0,8650.0,8650.0,19.89,320.99,55000.0,25.49,0.0,...,0,0,0,0,0,1,0,0,0,0
5,68506798.0,73396623.0,23000.0,23000.0,23000.0,8.49,471.77,64000.0,18.28,0.0,...,0,0,0,0,0,1,0,0,0,0
6,68566886.0,73456723.0,29900.0,29900.0,29900.0,12.88,678.49,65000.0,21.77,0.0,...,0,0,0,0,0,1,0,0,0,0
7,68577849.0,73467703.0,18000.0,18000.0,18000.0,11.99,400.31,112000.0,8.68,0.0,...,0,0,0,0,0,1,0,0,0,0
8,66310712.0,71035433.0,35000.0,35000.0,35000.0,14.85,829.90,110000.0,17.06,0.0,...,0,0,0,0,0,1,0,0,0,0
9,68476807.0,73366655.0,10400.0,10400.0,10400.0,22.45,289.91,104433.0,25.37,1.0,...,0,0,0,0,0,1,0,0,0,0


## Random Forest

In [33]:
start_time = time.time()
rfc = ensemble.RandomForestClassifier()
x = y2015.drop('loan_status', 1)
y = y2015['loan_status']
x = pd.get_dummies(x)
x = x.dropna(axis=1)

print(cross_val_score(rfc, x, y, cv=10))

print("--- %s seconds ---" % (time.time() - start_time))

[0.98033768 0.98067013 0.98171499 0.98162    0.9728568  0.97630017
 0.95708756 0.98074046 0.98012207 0.98050159]
--- 250.46189737319946 seconds ---


## Decision Tree

In [20]:
z2015 = y2015.copy()

In [21]:
c = set(z2015.columns)

In [22]:
ob = set(z2015.select_dtypes(exclude = 'float').columns)

In [23]:
ob

{'application_type',
 'emp_length',
 'grade',
 'home_ownership',
 'initial_list_status',
 'issue_d',
 'last_credit_pull_d',
 'last_pymnt_d',
 'loan_status',
 'next_pymnt_d',
 'purpose',
 'pymnt_plan',
 'term',
 'title',
 'verification_status',
 'verification_status_joint'}

In [24]:
flo = list(c - ob)

In [25]:
flo

['total_rec_prncp',
 'open_il_12m',
 'mths_since_rcnt_il',
 'percent_bc_gt_75',
 'policy_code',
 'revol_bal',
 'installment',
 'inq_last_12m',
 'total_pymnt',
 'mths_since_recent_inq',
 'acc_open_past_24mths',
 'collection_recovery_fee',
 'max_bal_bc',
 'open_rv_12m',
 'num_tl_90g_dpd_24m',
 'delinq_2yrs',
 'num_il_tl',
 'bc_open_to_buy',
 'loan_amnt',
 'total_pymnt_inv',
 'total_rev_hi_lim',
 'mths_since_last_delinq',
 'last_pymnt_amnt',
 'num_bc_tl',
 'int_rate',
 'num_rev_tl_bal_gt_0',
 'total_bal_il',
 'mths_since_last_record',
 'mo_sin_rcnt_tl',
 'total_bal_ex_mort',
 'id',
 'total_bc_limit',
 'tot_cur_bal',
 'out_prncp',
 'num_rev_accts',
 'collections_12_mths_ex_med',
 'annual_inc',
 'total_cu_tl',
 'total_acc',
 'open_il_6m',
 'mths_since_recent_bc',
 'inq_last_6mths',
 'all_util',
 'mths_since_recent_bc_dlq',
 'mo_sin_old_il_acct',
 'num_tl_op_past_12m',
 'mths_since_last_major_derog',
 'mths_since_recent_revol_delinq',
 'num_sats',
 'mo_sin_rcnt_rev_tl_op',
 'dti',
 'open_rv_

In [26]:
ob = list(ob)

In [27]:
for name in flo:
    z2015[name] = pd.to_numeric(y2015[name], errors='coerce')

In [94]:
for col in c:
    print(col)

id
member_id
loan_amnt
funded_amnt
funded_amnt_inv
term
int_rate
installment
grade
emp_length
home_ownership
annual_inc
verification_status
issue_d
loan_status
pymnt_plan
purpose
title
dti
delinq_2yrs
inq_last_6mths
mths_since_last_delinq
mths_since_last_record
open_acc
pub_rec
revol_bal
total_acc
initial_list_status
out_prncp
out_prncp_inv
total_pymnt
total_pymnt_inv
total_rec_prncp
total_rec_int
total_rec_late_fee
recoveries
collection_recovery_fee
last_pymnt_d
last_pymnt_amnt
next_pymnt_d
last_credit_pull_d
collections_12_mths_ex_med
mths_since_last_major_derog
policy_code
application_type
annual_inc_joint
dti_joint
verification_status_joint
acc_now_delinq
tot_coll_amt
tot_cur_bal
open_acc_6m
open_il_6m
open_il_12m
open_il_24m
mths_since_rcnt_il
total_bal_il
il_util
open_rv_12m
open_rv_24m
max_bal_bc
all_util
total_rev_hi_lim
inq_fi
total_cu_tl
inq_last_12m
acc_open_past_24mths
avg_cur_bal
bc_open_to_buy
bc_util
chargeoff_within_12_mths
delinq_amnt
mo_sin_old_il_acct
mo_sin_old_rev_

In [29]:
le = preprocessing.LabelEncoder()

In [30]:
for column in ob:
    #if z2015[column].dtype != type(float):
    z2015[column] = le.fit_transform(z2015[column].astype(str))

In [31]:
dtypes = pd.DataFrame(z2015.dtypes)
for val in dtypes.iloc[:,0]:
    print(val)

float64
float64
float64
float64
float64
int64
float64
float64
int64
int64
int64
float64
int64
int64
int64
int64
int64
int64
float64
float64
float64
float64
float64
float64
float64
float64
float64
int64
float64
float64
float64
float64
float64
float64
float64
float64
float64
int64
float64
int64
int64
float64
float64
float64
int64
float64
float64
int64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64


In [32]:
#x_train,x_test,y_train,y_test = cross_validation.train_test_split(x,y,test_size=0.4,random_state=0)
#depth = []
x = z2015.drop('loan_status', 1)
y = z2015['loan_status']
x = x.dropna(axis=1)

for i in range(1,103):
    start_time = time.time()

    clf = tree.DecisionTreeClassifier(criterion ='entropy',max_features=i)
    # Perform 10-fold cross validation 
    scores = cross_val_score(estimator=clf, X=x, y=y, cv=10)
    print("--- %s seconds ---" % (time.time() - start_time))
    print('#'+ str(i) +' try with score\n' + str(scores) + '\n' + 40*'-')
    print('with mean ' + str(scores.mean())+ '\n')
    #depth.append((i,scores.mean()))
#print(depth)

--- 15.591936349868774 seconds ---
#1 try with score
[0.77374083 0.83664601 0.8891976  0.85671202 0.74481121 0.86829732
 0.84098411 0.79636182 0.8496687  0.76599535]
----------------------------------------
with mean 0.8222414961694723

--- 19.799615144729614 seconds ---
#2 try with score
[0.89043243 0.92360666 0.92336919 0.91840612 0.85473759 0.89102351
 0.8743499  0.88548766 0.88215736 0.89645181]
----------------------------------------
with mean 0.8940022234682985

--- 24.259324073791504 seconds ---
#3 try with score
[0.7996248  0.93951699 0.89971741 0.92783358 0.9315602  0.85276656
 0.89182835 0.94333753 0.85486974 0.899943  ]
----------------------------------------
with mean 0.894099816799584

--- 28.857722282409668 seconds ---
#4 try with score
[0.8616751  0.92543516 0.92068581 0.95951177 0.73685585 0.90054619
 0.95145931 0.93611817 0.93300401 0.93829858]
----------------------------------------
with mean 0.9063589942436666

--- 32.35572600364685 seconds ---
#5 try with score
[

--- 153.92926216125488 seconds ---
#36 try with score
[0.95977298 0.95454869 0.95659091 0.96295505 0.91671812 0.95853716
 0.92802014 0.95409532 0.94739592 0.9515746 ]
----------------------------------------
with mean 0.9490208898930806

--- 157.91980361938477 seconds ---
#37 try with score
[0.94683099 0.95663841 0.95381254 0.96620835 0.92571836 0.95943956
 0.95274169 0.95540146 0.9405562  0.94967463]
----------------------------------------
with mean 0.9507022204226592

--- 156.37769222259521 seconds ---
#38 try with score
[0.95331386 0.95917931 0.95571228 0.96521099 0.94196153 0.95594871
 0.95635137 0.96067349 0.94889211 0.94998338]
----------------------------------------
with mean 0.9547227029642432

--- 158.3495111465454 seconds ---
#39 try with score
[0.95827693 0.95775451 0.95419249 0.9606991  0.90341962 0.95169793
 0.95364411 0.95445154 0.94319234 0.94532846]
----------------------------------------
with mean 0.9482657042590281

--- 161.06992435455322 seconds ---
#40 try with s

--- 278.79714798927307 seconds ---
#71 try with score
[0.95117665 0.95319513 0.9606991  0.96055662 0.9075279  0.94918072
 0.92236814 0.94749341 0.95535184 0.94889089]
----------------------------------------
with mean 0.945644041446555

--- 283.2767598628998 seconds ---
#72 try with score
[0.95032177 0.95459619 0.96067536 0.96072285 0.9097839  0.94861078
 0.93001496 0.94160393 0.95361816 0.94741842]
----------------------------------------
with mean 0.9457366322272616

--- 285.4592328071594 seconds ---
#73 try with score
[0.94951438 0.95442996 0.96086533 0.96169647 0.90771788 0.94903823
 0.92227315 0.94110523 0.95452063 0.94701468]
----------------------------------------
with mean 0.9448175931912223



ValueError: max_features must be in (0, n_features]

In [4]:
#I forgot to calculate the mean score for Random Forest, let's make it up here
RFscore = [0.98033768,0.98067013,0.98171499,0.98162,0.9728568,0.97630017,0.95708756,0.98074046,0.98012207,0.98050159]
sum(RFscore)/len(RFscore)

0.9771951449999998

It is pretty clear that Random Forest works much efficient and accurate than just Decision Tree. The 73rd calculation of Decision Tree result in longer time 285.45s and lower mean 0.9448. Moreover,I did much more feature engineering for Decision Tree than Random Forest, such as making sure all float is numeric, not mixing data type and using LabelEncoder to transform all object columns into str data type. All of these are not required for Random Forest for this specific case.